# Relative abundance changes

Single cell protein abundances are compared between WT and deletion backgrounds to calculate the relative protein abundances.

In [1]:
## logging functions
import logging
## system functions
from os.path import dirname
from os.path import splitext
import sys
## system functions from roux
from roux.lib.io import backup
from roux.lib.io import read_table
## workflow functions from roux
from roux.workflow.io import read_metadata
sys.path.append('..')

In [2]:
## parameters
metadata_path='../config/metadata.yaml'
kernel=None
force=False
test=True

In [3]:
## inferred parameters
metadata=read_metadata(metadata_path,inputs=None if not test else {'version':{'number':'test'}},)
metadata['dataset']=read_metadata(metadata['dataset_config_path'],config_base=dict(species_name=metadata['species_name'],path=metadata['dataset_path'],),)
### output
output_dir_path=metadata['processed']['abundance']+'abundance_change'
logging.info(f"Output directory: {output_dir_path}")
## backup old files if overwriting (force is True)
if force: backup(output_dir_path,dirname(output_dir_path),test=not force,)
## misc.
if kernel is None:
    kernel=metadata['kernels']['default']

## Calculate the relative change in abundance

In [4]:
parameters_list=[]
for k in ['paralogs','controls']:
    parameters_list.append(
        dict(
            ## parameters
            input_path=metadata['abundance']['cells'][k],
            output_path=metadata['abundance_change']['genes'][k],
            column_abundance='protein abundance',
            # columns in the table to groupby with. e.g. gene or gene and replicate
            columns_groupby=[
             'pairs',
             'gene symbol query',
            ],
            cutoff_qvalue=metadata['processed']['settings']['max_q_abundance_change']['value'], # cutoff to be applied on the p-values associated with the abundance changes
            cutoff_pvalue=None, # cutoff to be applied on the p-values associated with the abundance changes
            cutoff_log_fold_change=metadata['processed']['settings']['min_abundance_change_lfc']['value'], # cutoff to be applied on the log-fold-change scores
    ))

In [6]:
from roux.workflow.task import run_tasks
_=run_tasks(
    input_notebook_path='32_script_abundance_change.ipynb',
    kernel=kernel,
    parameters_list=parameters_list,
    force=True,
   )

## Plots

### Bar plots to show the gene-wise abundances and differences

In [6]:
%reset_selective -f "^df.*"

In [7]:
## Relative abundance change
df01=read_table(metadata['abundance_change'])
df01.head(1)

In [11]:
df1=df01.query(expr="`pairs sorted` == 'CUE1-CUE4'")

In [ ]:
from modules.tools.plot import plot_relative_abundance_change
(df01
.groupby('pairs sorted')
.progress_apply(lambda df: 
plot_relative_abundance_change(
    df,
    outp=splitext(metadata['abundance_change'])[0]+f"_plots/{df.name}",
)
)
)